#Text Mining Project 2021-2022<br/>

Matteo Cesaro - 867350 - m.cesaro1@campus.unimib.it<br/>
Francesco Martinelli - 873685 - f.martinelli21@campus.unimib.it<br/>
Cristiano Ruttico - 809360 - c.ruttico@campus.unimib.it<br/>

# Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


#Importing Librerie

In [ ]:
# Manage file
from time import time
from shutil import copyfile
import tarfile
import os
import re

#Basic library
import pandas as pd
import numpy as np

#NLTK libraries normalization
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import collections

#NLTK libaries Stemmin, Token and Lemma
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet') 
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

#Worcloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


#Gestione File

In [ ]:
copyfile('/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/Dataset/aclImdb_v1.tar', 'aclImdb_v1.tar')

'aclImdb_v1.tar'

In [ ]:
t0 = time()
tar = tarfile.open('aclImdb_v1.tar')
tar.extractall()
tar.close()
print("File transfer completed in %0.3f seconds" % (time() - t0))

File transfer completed in 25.381 seconds


# NEGATIVE-TEST file

Leggo ora tutti i file e li inserisco in una lista. Salvo inoltre i riferimenti alla categoria positivo o negativo associando il valore 0 a positivo e 1 a negativo.

In [ ]:
lst = []
label = []

In [ ]:
it = os.listdir("/content/aclImdb/test/neg")
len(it)
it.sort()
root_dir = "/content/aclImdb/test/neg/"

In [ ]:
for name_dcm in it:
  path = root_dir + name_dcm
  with open(path) as f:
    lines = f.readlines()
    lst.append(lines)
    label.append(1)

In [ ]:
lst[0:5]

[["Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in."],
 ["This is an example of why the majority of action films are the same. Generic and boring, t

In [ ]:
label[0]

1

# NEGATIVE/TRAIN file

In [ ]:
it = os.listdir("/content/aclImdb/train/neg")
len(it)
it.sort()
root_dir = "/content/aclImdb/train/neg/"

In [ ]:
for name_dcm in it:
  path = root_dir + name_dcm
  with open(path) as f:
    lines = f.readlines()
    lst.append(lines)
    label.append(1)

In [ ]:
len(lst)

25000

# POSITIVE/TEST file

In [ ]:
it = os.listdir("/content/aclImdb/test/pos")
len(it)
it.sort()
root_dir = "/content/aclImdb/test/pos/"

In [ ]:
for name_dcm in it:
  path = root_dir + name_dcm
  with open(path) as f:
    lines = f.readlines()
    lst.append(lines)
    label.append(0)

In [ ]:
len(lst)

37500

# POSITIVE/TRAIN file

In [ ]:
it = os.listdir("/content/aclImdb/train/pos")
len(it)
it.sort()
root_dir = "/content/aclImdb/train/pos/"

In [ ]:
for name_dcm in it:
  path = root_dir + name_dcm
  with open(path) as f:
    lines = f.readlines()
    lst.append(lines)
    label.append(0)

In [ ]:
len(lst)

50000

In [ ]:
len(label)

50000

# DATASET

Creo il primo dataset con i testi delle reviews e le label associate.

In [ ]:
df = pd.DataFrame (lst)

In [ ]:
df['label'] = label

In [ ]:
df.head()

,0,label
0,Once again Mr. Costner has dragged out a movie...,1
1,This is an example of why the majority of acti...,1
2,"First of all I hate those moronic rappers, who...",1
3,Not even the Beatles could write songs everyon...,1
4,Brass pictures (movies is not a fitting word f...,1


In [ ]:
df.rename(columns={ df.columns[0]: "text" }, inplace = True)

In [ ]:
#df.to_csv('/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/Dataset/df_tot')

# Preprocessing

Applico la normalizzazione alle reviews presenti nel dataset

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/Dataset/df_tot', index_col=[0])

In [ ]:
df.head()

,text,label
0,Once again Mr. Costner has dragged out a movie...,1
1,This is an example of why the majority of acti...,1
2,"First of all I hate those moronic rappers, who...",1
3,Not even the Beatles could write songs everyon...,1
4,Brass pictures (movies is not a fitting word f...,1


In [ ]:
df.columns

Index(['text', 'label'], dtype='object')

Case Folding

In [ ]:
df.text = df.text.str.lower()

Remove URL

In [ ]:
df.text = df.text.str.replace(r'https?://\S+|www\.\S+', '', regex=True)

Remove Emoji

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
df.text = df.text.apply(lambda x: remove_emoji(x))

Remove number

In [ ]:
df.text = df.text.str.replace(r'\d+', '', regex=True)

Remove stopwords

In [ ]:
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in STOPWORDS])

In [ ]:
df.text = df.text.apply(lambda text: remove_stopwords(text))

Remove punctuaction

In [ ]:
PUNCTUATIONS = string.punctuation.replace('#','')

def remove_punctuation(text):
    trans = str.maketrans(dict.fromkeys(PUNCTUATIONS, ' '))
    return text.translate(trans)

In [ ]:
df.text = df.text.apply(lambda x: remove_punctuation(x))

Remove whitespace

In [ ]:
def remove_whitespaces(text):
    return " ".join(text.split())

In [ ]:
df.text = df.text.apply(lambda x: remove_whitespaces(x))

Lemmatizzazione

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
df.head()

,text,label
0,mr costner dragged movie far longer necessary ...,1
1,example majority action films same generic bor...,1
2,first hate moronic rappers could nt act gun pr...,1
3,even beatles could write songs everyone liked ...,1
4,brass pictures movies fitting word them really...,1


In [ ]:
#df.to_csv('/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/Dataset/df_tot_preproc')

# Token, lemma and stem

Applico la tokenizzazione e in seguito la lemmatizzazione e lo stemming per poter provare eventuali confronti tra queste due tecniche.

In [ ]:
df_preproc = pd.read_csv('/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/Dataset/df_tot_preproc', index_col=[0] )

In [ ]:
df_preproc.head()

,text,label
0,mr costner dragged movie far longer necessary ...,1
1,example majority action films same generic bor...,1
2,first hate moronic rappers could nt act gun pr...,1
3,even beatles could write songs everyone liked ...,1
4,brass pictures movies fitting word them really...,1


In [ ]:
reviews = df_preproc['text'].tolist()
type(reviews)

list

In [ ]:
len(reviews)

50000

In [ ]:
tokenizen_lst = []
for i in reviews:
  reviews_split = wordpunct_tokenize(i)
  tokenizen_lst.append(reviews_split)

In [ ]:
len(tokenizen_lst)

50000

In [ ]:
print(tokenizen_lst[0])

['mr', 'costner', 'dragged', 'movie', 'far', 'longer', 'necessary', 'aside', 'terrific', 'sea', 'rescue', 'sequences', 'care', 'characters', 'us', 'ghosts', 'closet', 'costner', 's', 'character', 'realized', 'early', 'on', 'forgotten', 'much', 'later', 'time', 'care', 'character', 'really', 'care', 'cocky', 'overconfident', 'ashton', 'kutcher', 'problem', 'comes', 'kid', 'thinks', 'he', 's', 'better', 'anyone', 'else', 'around', 'shows', 'signs', 'cluttered', 'closet', 'obstacle', 'appears', 'winning', 'costner', 'finally', 'well', 'past', 'half', 'way', 'point', 'stinker', 'costner', 'tells', 'us', 'kutcher', 's', 'ghosts', 'told', 'kutcher', 'driven', 'best', 'prior', 'inkling', 'foreshadowing', 'magic', 'here', 'could', 'keep', 'turning', 'hour', 'in']


In [ ]:
lemma_text = []
for line in tokenizen_lst:
  piccola_lista = []
  for word in line:
    word_lmtz = wordnet_lemmatizer.lemmatize(word)
    piccola_lista.append(word_lmtz)
  lemma_text.append(piccola_lista)

In [ ]:
len(lemma_text)

50000

In [ ]:
print(lemma_text[0])

['mr', 'costner', 'dragged', 'movie', 'far', 'longer', 'necessary', 'aside', 'terrific', 'sea', 'rescue', 'sequence', 'care', 'character', 'u', 'ghost', 'closet', 'costner', 's', 'character', 'realized', 'early', 'on', 'forgotten', 'much', 'later', 'time', 'care', 'character', 'really', 'care', 'cocky', 'overconfident', 'ashton', 'kutcher', 'problem', 'come', 'kid', 'think', 'he', 's', 'better', 'anyone', 'else', 'around', 'show', 'sign', 'cluttered', 'closet', 'obstacle', 'appears', 'winning', 'costner', 'finally', 'well', 'past', 'half', 'way', 'point', 'stinker', 'costner', 'tell', 'u', 'kutcher', 's', 'ghost', 'told', 'kutcher', 'driven', 'best', 'prior', 'inkling', 'foreshadowing', 'magic', 'here', 'could', 'keep', 'turning', 'hour', 'in']


In [ ]:
stem_text = []
for line in tokenizen_lst:
  piccola_lista = []
  for word in line:
    word_lmtz = porter.stem(word)
    piccola_lista.append(word_lmtz)
  stem_text.append(piccola_lista)

In [ ]:
df_preproc['tokenized'] = tokenizen_lst

In [ ]:
df_preproc['lemmatized'] = lemma_text

In [ ]:
df_preproc['stemmed'] = stem_text

In [ ]:
df_preproc.head()

,text,label,tokenized,lemmatized,stemmed
0,mr costner dragged movie far longer necessary ...,1,"[mr, costner, dragged, movie, far, longer, nec...","[mr, costner, dragged, movie, far, longer, nec...","[mr, costner, drag, movi, far, longer, necessa..."
1,example majority action films same generic bor...,1,"[example, majority, action, films, same, gener...","[example, majority, action, film, same, generi...","[exampl, major, action, film, same, gener, bor..."
2,first hate moronic rappers could nt act gun pr...,1,"[first, hate, moronic, rappers, could, nt, act...","[first, hate, moronic, rapper, could, nt, act,...","[first, hate, moron, rapper, could, nt, act, g..."
3,even beatles could write songs everyone liked ...,1,"[even, beatles, could, write, songs, everyone,...","[even, beatles, could, write, song, everyone, ...","[even, beatl, could, write, song, everyon, lik..."
4,brass pictures movies fitting word them really...,1,"[brass, pictures, movies, fitting, word, them,...","[brass, picture, movie, fitting, word, them, r...","[brass, pictur, movi, fit, word, them, realli,..."


In [ ]:
print(df_preproc.dtypes)

text          object
label          int64
tokenized     object
lemmatized    object
stemmed       object
dtype: object


In [ ]:
#df_preproc.to_csv('/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/Dataset/df_preproc_nltk_punct_based')

# CARICAMENTO DF PREPROC LEMMA STEMMA

Operazioni necessarie per poter lavorare sulle reviews.

In [ ]:
df_tot = pd.read_csv('/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/Dataset/df_preproc_nltk', index_col=[0] )

In [ ]:
df_tot.head()

,text,label,tokenized,lemmatized,stemmed
0,mr costner dragged movie far longer necessary ...,1,"['mr', 'costner', 'dragged', 'movie', 'far', '...","['mr', 'costner', 'dragged', 'movie', 'far', '...","['mr', 'costner', 'drag', 'movi', 'far', 'long..."
1,example majority action films same generic bor...,1,"['example', 'majority', 'action', 'films', 'sa...","['example', 'majority', 'action', 'film', 'sam...","['exampl', 'major', 'action', 'film', 'same', ..."
2,first hate moronic rappers could nt act gun pr...,1,"['first', 'hate', 'moronic', 'rappers', 'could...","['first', 'hate', 'moronic', 'rapper', 'could'...","['first', 'hate', 'moron', 'rapper', 'could', ..."
3,even beatles could write songs everyone liked ...,1,"['even', 'beatles', 'could', 'write', 'songs',...","['even', 'beatles', 'could', 'write', 'song', ...","['even', 'beatl', 'could', 'write', 'song', 'e..."
4,brass pictures movies fitting word them really...,1,"['brass', 'pictures', 'movies', 'fitting', 'wo...","['brass', 'picture', 'movie', 'fitting', 'word...","['brass', 'pictur', 'movi', 'fit', 'word', 'th..."


In [ ]:
df_tot["tokenized"] = df_tot["tokenized"].apply(lambda x: x.replace(",", ""))
df_tot["tokenized"] = df_tot["tokenized"].apply(lambda x: x.replace("'", ""))
df_tot["tokenized"] = df_tot["tokenized"].apply(lambda x: x.replace("[", ""))
df_tot["tokenized"] = df_tot["tokenized"].apply(lambda x: x.replace("]", ""))

In [ ]:
df_tot["lemmatized"] = df_tot["lemmatized"].apply(lambda x: x.replace(",", ""))
df_tot["lemmatized"] = df_tot["lemmatized"].apply(lambda x: x.replace("'", ""))
df_tot["lemmatized"] = df_tot["lemmatized"].apply(lambda x: x.replace("[", ""))
df_tot["lemmatized"] = df_tot["lemmatized"].apply(lambda x: x.replace("]", ""))

In [ ]:
df_tot["stemmed"] = df_tot["stemmed"].apply(lambda x: x.replace(",", ""))
df_tot["stemmed"] = df_tot["stemmed"].apply(lambda x: x.replace("'", ""))
df_tot["stemmed"] = df_tot["stemmed"].apply(lambda x: x.replace("[", ""))
df_tot["stemmed"] = df_tot["stemmed"].apply(lambda x: x.replace("]", ""))

In [ ]:
df_tot.head()

,text,label,tokenized,lemmatized,stemmed
0,mr costner dragged movie far longer necessary ...,1,mr costner dragged movie far longer necessary ...,mr costner dragged movie far longer necessary ...,mr costner drag movi far longer necessari asid...
1,example majority action films same generic bor...,1,example majority action films same generic bor...,example majority action film same generic bori...,exampl major action film same gener bore there...
2,first hate moronic rappers could nt act gun pr...,1,first hate moronic rappers could nt act gun pr...,first hate moronic rapper could nt act gun pre...,first hate moron rapper could nt act gun press...
3,even beatles could write songs everyone liked ...,1,even beatles could write songs everyone liked ...,even beatles could write song everyone liked a...,even beatl could write song everyon like altho...
4,brass pictures movies fitting word them really...,1,brass pictures movies fitting word them really...,brass picture movie fitting word them really s...,brass pictur movi fit word them realli somewha...


In [ ]:
text = " ".join(review for review in df_tot.tokenized)

wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

NameError: ignored

In [ ]:
#wordcloud.to_file("/content/gdrive/MyDrive/UNIMIB/TextMiningAndSearch/Text mining/wordcloud.png")